_Este material es adicional al contenido mínimo del curso. Se deja aquí para quien quiera explorar cómo implementar los algoritmos de recorrido de árboles BFS y DFS pero utilizando herramientas de Programación Funcional. Esto no será evaluado a lo largo del curso, pero siempre es entretenido conocer más._

# Recorrido de árboles: Versión funcional

Un caso de uso frecuente que podríamos necesitar es recorrer un árbol según algún orden (BFS o DFS) y realizar algún trabajo o revisión sobre los nodos en cada paso. Si bien, el código antes presentado en el material mínimo se puede alterar fácilmente dependiendo de la sitaución, requiere re-escribir el código de recorrido para cada caso particular.

Lo conveniente sería crear algún tipo de interfaz para recorrer un árbol que nos permita obtener el `nodo_actual` en cada paso y nos permita trabajar con el. Para esto, utilizaremos los conceptos de estructuras **iterables** y **generadores** de programación funcional en Python.

Primero, se vuelve a crear la clase `Arbol` base, **identica** a la del material de esta semana:

In [1]:
# textwrap tiene varias funciones convenientes para el manejo de strings
from textwrap import indent

class Arbol:
    """
    Esta clase representa un árbol
    
    La estructura es recursiva, de manera que cada nodo es la raíz de un sub-árbol.
    Los nodos hijos pueden ser guardados en una estructura, como lista o diccionario.
    En este ejemplo, los nodos hijos serán guardados en un diccionario.
    """

    def __init__(self, id_nodo, valor=None, padre=None):
        """
        Inicializa la estructura básica del árbol.
        
        Tiene un identificador propio, la referencia a su padre, el valor almacenado
        y una estructura con sus hijos.
        """
        self.id_nodo = id_nodo
        self.padre = padre
        self.valor = valor
        self.hijos = {}
        

    def obtener_nodo(self, id_nodo):
        """
        Obtiene el nodo con el id dado, en forma recursiva
        
        A partir del nodo actual, buscamos el nodo 'id_nodo' entre sus hijos
        y lo retornamos si existe.
        """
        # Caso base: ¡Lo encontramos!
        if self.id_nodo == id_nodo:
            return self

        # Buscamos recursivamente entre los hijos
        for hijo in self.hijos.values():
            nodo = hijo.obtener_nodo(id_nodo)
            # Si lo encontró, lo retornamos
            if nodo:
                return nodo
        
        # Si no lo encuentra, retorna None
        return None


    def agregar_nodo(self, id_nodo, valor, id_padre):
        """Agrega un nodo con el id y valor dado, como hijo del nodo con el id 'id_padre'"""
        # Primero, tenemos que encontrar al padre
        padre = self.obtener_nodo(id_padre)
        # En caso de que el padre no exista no hacemos nada
        if padre is None:
            return
        
        # Creamos el nodo
        # Nos aseguramos de que el nodo nuevo sea del mismo tipo que la raíz
        # Esto lo ocuparemos cuando heredemos de este árbol
        nodo = type(self)(id_nodo, valor, padre)
        # Agregamos el nodo como hijo de su padre
        padre.hijos[id_nodo] = nodo
        
        
    def __repr__(self):
        """
        Entrega una representación del árbol, en forma recursiva.
        
        Para ello, tenemos que pedir la representación de cada hijo recursivamente. 
        Esto nos lleva a recorrer todos los nodos del árbol.
        """
        # Texto de este nodo.
        # Si el nodo es hoja, se avisa de ello.
        # Si el nodo no es hoja, se deja un salto de línea para poder nombrar a los hijos.
        if len(self.hijos) > 0:
            texto = f"id: {self.id_nodo}, valor: {self.valor}\n"
        else:
            texto = f"id: {self.id_nodo}, valor: {self.valor}, nodo hoja"

        # Extrae el repr a cada hijo, en forma recursiva.
        texto_hijos = [repr(hijo) for hijo in self.hijos.values()]
        
        # Indentamos cada línea del texto de los hijos con dos espacios.
        # Esto es para que se note el nivel del nodo.
        texto_hijos = [indent(x, "  ")  for x in texto_hijos]
        
        return texto + "\n".join(texto_hijos)


## BFS

En el siguiente _snippet_, hacemos que el árbol sea un iterador (y por lo tanto iterable). Una forma de hacerlo, es definiendo una función generadora para `__iter__`. La idea es que al iterar el árbol, nos entregue los nodos en orden BFS. Para detalles sobre iteradores, iterables y generadores, pueden revisar el material de programación funcional.

In [2]:
from collections import deque

class ArbolBFS(Arbol):
    
    def __iter__(self):
        """Itera el árbol según BFS"""
        # Utilizamos una cola para almacenar los nodos por visitar
        cola = deque()
        # El primer nodo a visitar será la raíz
        cola.append(self)
        
        # Mientras queden nodos por visitar en la cola
        while len(cola) > 0:
            # Extraemos el primero de la cola
            nodo = cola.popleft()
            
            # Lo visitamos y entrega
            yield nodo
            
            # Agregamos todos los nodos hijos a la cola
            for hijo in nodo.hijos.values():
                cola.append(hijo)

In [3]:
T = ArbolBFS(0, 10)
T.agregar_nodo(1, 8, 0)
T.agregar_nodo(3, 12, 0)
T.agregar_nodo(2, 9, 1)
T.agregar_nodo(4, 5, 3)
T.agregar_nodo(5, 14, 3)
T.agregar_nodo(6, 20, 3)
T.agregar_nodo(8, 4, 2)
T.agregar_nodo(7, 8, 4)
T.agregar_nodo(9, 15, 6)
T.agregar_nodo(10, 6, 6)

Ahora, como es un iterable, podemos realizar un `for` sobre el directamente, donde el orden de nodos sigue el orden BFS:

In [4]:
print("Recorrido EN AMPLITUD (BFS, o 'por niveles')")
for i, nodo in enumerate(T):
    print(f"Número de visita: {i} — visitando nodo con id: {nodo.id_nodo}")

Recorrido EN AMPLITUD (BFS, o 'por niveles')
Número de visita: 0 — visitando nodo con id: 0
Número de visita: 1 — visitando nodo con id: 1
Número de visita: 2 — visitando nodo con id: 3
Número de visita: 3 — visitando nodo con id: 2
Número de visita: 4 — visitando nodo con id: 4
Número de visita: 5 — visitando nodo con id: 5
Número de visita: 6 — visitando nodo con id: 6
Número de visita: 7 — visitando nodo con id: 8
Número de visita: 8 — visitando nodo con id: 7
Número de visita: 9 — visitando nodo con id: 9
Número de visita: 10 — visitando nodo con id: 10


## DFS

En el siguiente trozo de código, hacemos que un árbol sea iterable con el recorrido DFS.

In [5]:
from collections import deque

class ArbolDFS(Arbol):
    """Heredamos de la clase Arbol para hacerla iterable según el orden con DFS"""
    
    def __iter__(self):
        """Itera el árbol según DFS"""
        # Utilizamos un stack para almacenar los nodos por visitar
        stack = deque()
        # El primer nodo a visitar será la raíz
        stack.append(self)
        
        # Mientras queden nodos por visitar en el stack
        while len(stack) > 0:
            # Extraemos el que está en el tope del stack
            nodo = stack.pop()
            
            # Lo visitamos
            yield nodo
            
            # Agregamos todos los nodos hijos al stack
            for hijo in nodo.hijos.values():
                stack.append(hijo)

Poblamos el árbol con los datos usados en el ejemplo de la clase `Arbol`

In [6]:
T = ArbolDFS(0, 10)
T.agregar_nodo(1, 8, 0)
T.agregar_nodo(3, 12, 0)
T.agregar_nodo(2, 9, 1)
T.agregar_nodo(4, 5, 3)
T.agregar_nodo(5, 14, 3)
T.agregar_nodo(6, 20, 3)
T.agregar_nodo(8, 4, 2)
T.agregar_nodo(7, 8, 4)
T.agregar_nodo(9, 15, 6)
T.agregar_nodo(10, 6, 6)
T

id: 0, valor: 10
  id: 1, valor: 8
    id: 2, valor: 9
      id: 8, valor: 4, nodo hoja
  id: 3, valor: 12
    id: 4, valor: 5
      id: 7, valor: 8, nodo hoja
    id: 5, valor: 14, nodo hoja
    id: 6, valor: 20
      id: 9, valor: 15, nodo hoja
      id: 10, valor: 6, nodo hoja

In [7]:
print("Recorrido EN PROFUNDIDAD (DFS, o 'por ramas')")
for i, nodo in enumerate(T):
    print(f"Número de visita: {i} — visitando nodo con id: {nodo.id_nodo}")

Recorrido EN PROFUNDIDAD (DFS, o 'por ramas')
Número de visita: 0 — visitando nodo con id: 0
Número de visita: 1 — visitando nodo con id: 3
Número de visita: 2 — visitando nodo con id: 6
Número de visita: 3 — visitando nodo con id: 10
Número de visita: 4 — visitando nodo con id: 9
Número de visita: 5 — visitando nodo con id: 5
Número de visita: 6 — visitando nodo con id: 4
Número de visita: 7 — visitando nodo con id: 7
Número de visita: 8 — visitando nodo con id: 1
Número de visita: 9 — visitando nodo con id: 2
Número de visita: 10 — visitando nodo con id: 8


#### DFS recursivo

Es muy sencillo implementar DFS mediante una recursión, tal y como se muestra en el siguiente código:

In [8]:
from collections import deque

class ArbolDFSRecursivo(Arbol):
    """Heredamos de la clase Arbol para hacerla iterable según el orden con DFS recursivo"""
    
    def __iter__(self):
        """Itera el árbol según DFS recursivo"""
        # Visitamos el nodo actual
        yield self
        # Aplicamos esto recursivamente a cada hijo
        for hijo in self.hijos.values():
            yield from hijo

In [9]:
T = ArbolDFSRecursivo(0, 10)
T.agregar_nodo(1, 8, 0)
T.agregar_nodo(3, 12, 0)
T.agregar_nodo(2, 9, 1)
T.agregar_nodo(4, 5, 3)
T.agregar_nodo(5, 14, 3)
T.agregar_nodo(6, 20, 3)
T.agregar_nodo(8, 4, 2)
T.agregar_nodo(7, 8, 4)
T.agregar_nodo(9, 15, 6)
T.agregar_nodo(10, 6, 6)

In [10]:
print("Recorrido EN PROFUNDIDAD (DFS, o 'por ramas')")
for i, nodo in enumerate(T):
    print(f"Número de visita: {i} — visitando nodo con id: {nodo.id_nodo}")

Recorrido EN PROFUNDIDAD (DFS, o 'por ramas')
Número de visita: 0 — visitando nodo con id: 0
Número de visita: 1 — visitando nodo con id: 1
Número de visita: 2 — visitando nodo con id: 2
Número de visita: 3 — visitando nodo con id: 8
Número de visita: 4 — visitando nodo con id: 3
Número de visita: 5 — visitando nodo con id: 4
Número de visita: 6 — visitando nodo con id: 7
Número de visita: 7 — visitando nodo con id: 5
Número de visita: 8 — visitando nodo con id: 6
Número de visita: 9 — visitando nodo con id: 9
Número de visita: 10 — visitando nodo con id: 10
